In [6]:
import numpy as np

# We first import the time module to compute runtime
import time
start_time = time.time()


# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    Box_vertices = np.array(Box.vertices_list())
    return Box_vertices

# Minkowski sum function
def minkowski_pol(V1, V2):
    V3 = []
    for v1 in V1:
        for v2 in V2:
            n = v1.size
            v = np.zeros(n)
            for i in range(n):
                v[i] = v1[i] + v2[i]
            V3.append(v)
    # print(V3)
    V4 = np.array(V3)
    return V4

def convex_hull(V1, V2):
    V2 = np.append(V1,V2)
    return V2

def linear_trans(M, V):
    # V1 = P1.vertices_list()
    # V2 = P2.vertices_list()
    V1 = []
    for v in V:
        x =  np.matmul(M,v)
        x = np.array(x)
        V1.append(x)
        # print(V1)
    V2 = np.array(V1)
    return V2

# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 2: ")
exp_num = v
# Open the input file
if (int(v)==1):
    input_file = open(r"inputs_new/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs_new/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs_new/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs_new/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()

# Create the box
v = [0]*(2*n)
r = [ep]*n
zeros = [0]*n
rbox = zeros+r
Box = make_box(v,rbox)
# print('Box = ', Box.vertices())

# Converting K into K = [[I 0] [0 K]]
xn, un = n, u
K1 = [[0 for i in range(xn)]for j in range(xn)]
KI = np.identity(xn)
K1 = np.hstack((KI, K1))
K2 = [[0 for i in range(xn)] for j in range(un)]
K2 = np.hstack((K2,K))
K = Matrix(np.vstack((K1,K2)))

# At first, we define C = [A B]
C = Matrix(np.hstack((A,B)))

# Next, we have input P0 = [X0 U0]^T
P0 = np.array([np.hstack((x,u)) for x in X0 for u in U0])

# We define the matrix M as follows,
I = np.identity(xn)
M = np.array(np.vstack((I,I)))

# Opening a file to write the outputs
exp_string = "outputs/alg2_new/ex" + exp_num + ".csv"
f = open(exp_string, "a")
f.write('\nNew Experiment:\n')

f.write('Iteration, Num_Ver(Uc), Num_Ver(Xc), Runtime\n')
# Running algorithm 2
Pc = P0
iter = 50
for i in range(iter):
    try:
        # print('i = ', i, file=f)
        Xc = linear_trans(C,Pc)
        # print('Number_of_Cons(Xc) = ', len(list(Xc.Hrepresentation())), file=f)
        Xc_twice = linear_trans(M, Xc)
        # print('Number_of_Cons(Xc_twice) = ', len(list(Xc_twice.Hrepresentation())), file=f)
        Xh = minkowski_pol(Xc_twice,Box)
        # print('Number_of_Cons(Xh) = ', len(list(Xh.Hrepresentation())), file=f)
        Pc = linear_trans(K,Xh)
        # print('Number_of_Cons(Pc) = ', len(list(Pc.Hrepresentation())), file=f)
        # print('Vol(Xc) = ', float(Xc.volume()),  file=f)
        # print('Vol(Xh) = ', float(Xh.volume()),  file=f)
        # print('Vol(Pc) = ', float(Pc.volume()),  file=f)
        curr_time = (time.time()-start_time)*10**3
        msg = str((i+1, len(Pc), len(Xc), len(Xh), curr_time))
        msg = msg[1:-1]
        print(msg)
        f.write(msg+'\n')
    except KeyboardInterrupt:
        break

# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3

# msg = str((iter, len(list(Xc.Hrepresentation())), len(list(Xc.vertices())), len(list(Xc.vertices())[0]), total_time_in_ms, error_msg))
# msg = msg[1:-1]
# f.write(msg)

print("\nRuntime: ", total_time_in_ms, file = f)

# Close the file
f.close()

# Saving the plot
try:
    fig = Xc.plot()
    fig_string = "outputs/alg1/ex" + exp_num + ".png"
    fig.save(fig_string)
except:
    pass

# Plotting in notebook
Xc

1, 4, 4, 1534.9900722503662
2, 4, 4, 1535.5618000030518
3, 4, 4, 1535.8998775482178
4, 4, 4, 1536.2601280212402
5, 4, 4, 1536.72194480896
6, 4, 4, 1537.1708869934082
7, 4, 4, 1537.5678539276123
8, 4, 4, 1537.9769802093506
9, 4, 4, 1538.3899211883545
10, 4, 4, 1539.0450954437256
11, 4, 4, 1539.5939350128174
12, 4, 4, 1539.9978160858154
13, 4, 4, 1540.4269695281982
14, 4, 4, 1540.8918857574463
15, 4, 4, 1541.3038730621338
16, 4, 4, 1541.698932647705
17, 4, 4, 1542.205810546875
18, 4, 4, 1542.6297187805176
19, 4, 4, 1542.9317951202393
20, 4, 4, 1543.2367324829102
21, 4, 4, 1543.6158180236816
22, 4, 4, 1543.9558029174805
23, 4, 4, 1544.255018234253
24, 4, 4, 1544.5737838745117
25, 4, 4, 1544.8808670043945
26, 4, 4, 1545.26686668396
27, 4, 4, 1545.6299781799316
28, 4, 4, 1546.0069179534912
29, 4, 4, 1546.30708694458
30, 4, 4, 1546.651840209961
31, 4, 4, 1547.0430850982666
32, 4, 4, 1547.4419593811035
33, 4, 4, 1547.8248596191406
34, 4, 4, 1548.408031463623
35, 4, 4, 1548.7327575683594
36, 4

array([[ 37., -16.],
       [ 88., -37.],
       [ 60., -25.],
       [ 23., -10.]])